In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re 
from datetime import date, datetime
import numpy as np
import random
import copy
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
plt.style.use('seaborn')
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}

plt.rc('font', **font)
plt.rcParams.update({'font.size': 16})


# Task 1: Explore a small dataset
### Task 1A: Exploration

In [ ]:
df = pd.read_csv("data/ODI-2020.csv", sep=';')

# rename to normal column names for easier referencing later
df.columns=["program", "ml_course", "ir_course", "stat_course", "db_course", "gender",
                "chocolate", "birthday", "neighbors", "standed_up", "stress", "local_dm_comp", "random_number",
                "bedtime_yesterday", "happy1", "happy2"]
display(df)

### Clean

In [ ]:
# clean up open questions: master can be CLS/computational science/Computational Science/Master Computational Science, etc.
import string

df = pd.read_csv("data/ODI-2020.csv", sep=';')

# rename to normal column names for easier referencing later
df.columns=["program", "ml_course", "ir_course", "stat_course", "db_course", "gender",
                "chocolate", "birthday", "neighbors", "standed_up", "stress", "local_dm_comp", "random_number",
                "bedtime_yesterday", "happy1", "happy2"]
# display(df)


#print("Unique progams:", len(df.program.unique()))
#print("Changing everything to lowercase")
df.program = df.program.map(str.lower)

#print("Unique progams:", len(df.program.unique()))
#print("Deleting all punctuation")
# text = text.translate()

df.program = df.program.map(lambda s: s.translate(str.maketrans('','', string.punctuation)))
#print("Unique progams:", len(df.program.unique()))


#print("\n\n\n")

# # oude onderverdeling
# for index, row in df.iterrows():
#     if "computational" in row.program or "cls" in row.program:
#         row.program = "CLS"
#     elif "artificial" in row.program or "ai" in row.program:
#         row.program = "AI"
#     elif "computer science" in row.program or "cs" in row.program:
#         row.program = "CS"
#     elif "quantitative risk management" in row.program or "qrm" in row.program:
#         row.program = "QRM"

# nieuwe onderverdeling
for index, row in df.iterrows():
    if "computational" in row.program or "cls" in row.program or "computing" in row.program or "information" in row.program:
        row.program = "CLS"
    elif "artificial" in row.program or "ai" in row.program:
        row.program = "AI"
    elif "computer science" in row.program or "cs" in row.program or "parallel" in row.program or "data" in row.program:
        row.program = "CS"
    # finance studies
    elif "quantitative risk management" in row.program or "qrm" in row.program or "finance" in row.program or "business" in row.program:
        row.program = "QRM"
    else:
        row.program = "OTHER"
print("Unique progams:", len(df.program.unique()))
print(df.program.unique())

In [ ]:
display(df)

In [ ]:
# Clean birthday
df_new = df.copy()
df_new["age"] = np.nan

def calculateAge(birthDate): 
    today = date.today() 
    return today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day)) 

for index,row in df_new.iterrows():
    line = re.split("-|/|th|of|''|' ' ", row["birthday"])
    if len(line) == 1:
        line = line[0].split(".")

    if len(line) == 1:
        if line[0].startswith('199') or line[0].startswith('198'):
            year = int(line[0])

            if year > 1900 and year < 2010:
                date_str = date(year, 1, 1)
                age = calculateAge(date_str)
    
    elif len(line) == 3 and ' ' not in line:
        for i in range(len(line)): 
            if line[i] == "march ": 
                line[i] = 3
        line = [int(i) for i in line] 
        year, month, day = line[2], line[1], line[0]
        
        # For entries of '94' in stead of '1994'
        if year > 60 and year < 1900:
            year = int('19' + str(year))
            
         # Calculate date with year - month - day
        try:
            date_str = date(year, month, day)
            age = calculateAge(date_str)
            
            # Exclude wrong data
            if age == 0:
                continue
    
        except ValueError:
            date_str = np.nan
            age = np.nan
            continue
        
    # Add new column of 'age'
    df_new.birthday[index] = date_str
    df_new.age[index] = age

In [ ]:
# Further cleaning

# ml_course
# array(['no', 'yes', 'unknown'], dtype=object)
clean_ml = []
for nr in df_new['ml_course']:
    if nr == 'no':
        clean_ml.append(0)
    elif nr == 'yes':
        clean_ml.append(1)
    else:
        clean_ml.append(np.nan)
df_new['ml_course'] = clean_ml

# ir_course
# array(['unknown', '0', '1'], dtype=object) where 0 is no, 1 is yes
clean_ir = []
for ir in df_new['ir_course']:
    if ir == '0':
        clean_ir.append(0)
    elif ir == '1':
        clean_ir.append(1)
    else:
        clean_ir.append(np.nan)
df_new['ir_course'] = clean_ir

print(df_new['ir_course'].unique())

# stat_course
# array(['mu', 'sigma', 'unknown'], dtype=object) mu is yes, sigma is no
clean_stat = []
for st in df_new['stat_course']:
    if st == 'sigma':
        clean_stat.append(0)
    elif st == 'mu':
        clean_stat.append(1)
    else:
        clean_stat.append(np.nan)
df_new['stat_course'] = clean_stat

# db_course
# array(['ja', 'nee', 'unknown'], dtype=object)
clean_db = []
for db in df_new['db_course']:
    if db == 'nee':
        clean_db.append(0)
    elif db == 'ja':
        clean_db.append(1)
    else:
        clean_db.append(np.nan)
df_new['db_course'] = clean_db

# gender
# array(['female', 'male', 'unknown'], dtype=object)

# chocolate
# array(['fat', 'unknown', 'neither', 'I have no idea what you are talking about', 'slim'], dtype=object)

# neighbors
# convert values to integers if possible, else 'nan'
cleaned_neighbors = []
for nr in df_new['neighbors']:
    try:
        cleaned_neighbors.append(int(nr))
    except:
        cleaned_neighbors.append(np.nan)

# replace old column by cleaned one
df_new['neighbors'] = cleaned_neighbors

# standed_up = array(['no', 'unknown', 'yes'], dtype=object)

# stress
cleaned_stress = []
for nr in df_new['stress']:
    try:
        new = int(nr)
        if new < 0 or new > 100:
            cleaned_stress.append(np.nan)
        else:
            cleaned_stress.append(new)
    except:
        cleaned_stress.append(np.nan)

# replace old column by cleaned one
df_new['stress'] = cleaned_stress

# local_dm_comp
# TODO

# random_number
# Note: check for outliers, they are highly influencing the mean
cleaned_random = []
for index, row in df.iterrows():
    try:
        cleaned_random.append(int(row['random_number']))
    except:
        cleaned_random.append(np.nan)

# replace old column by cleaned one
df_new['random_number'] = cleaned_random


# bedtime_yesterday
# for now only accepting completely numeric (+ ":") entries, that is 151 of 280 entries
# first convert entries to strings
cleaned_bedtime_strings = []
for index,row in df_new.iterrows():
    incorrect = False
    value = str(row['bedtime_yesterday'])
    for letter in value:
        if letter != ':':
            try:
                int(letter)
            except:
                incorrect = True
    if not incorrect:
        cleaned_bedtime_strings.append(value)
    else:
        cleaned_bedtime_strings.append(np.nan)

# convert strings to time objects, else make it nan
time_objects = []
c = 0
for item in cleaned_bedtime_strings:
    if item != item:
        time_objects.append(np.nan)
    elif ':' in item and len(item) == 5:
        try:
            time_objects.append(datetime.strptime(item, '%H:%M').time())
            c += 1
        except:
            time_objects.append(np.nan)
    else:
        time_objects.append(np.nan)

    
# replace old column by cleaned one
df_new['bedtime_yesterday'] = time_objects

    
# happy1 & happy 2 
# voeg columns 'sun' en 'food' toe aan dataframe, 
# sun: 1 = 'sun' of 'weather' komt voor in happy1 of happy2, 0 = komt niet voor
# food: 1 = 'food' of 'choc' komt voor in happy1 of happy2, 0 = komt niet voor
# ik heb ook overwogen te checken of 'corona' veel genoemd wordt, maar dat werd maar 4x genoemd dus niet erin gedaan
sun = []
food = []
c = 0
for index,row in df_new.iterrows():
    if 'sun' in str(row['happy2']).lower() or 'weather' in str(row['happy2']).lower() or 'sun' in str(row['happy1']).lower() or 'weather' in str(row['happy1']).lower():
        sun.append(1)
    else:
        sun.append(0)
    if 'food' in str(row['happy1']).lower() or 'food' in str(row['happy2']).lower() or 'choc' in str(row['happy1']).lower() or 'choc' in str(row['happy2']).lower():
        food.append(1)
    else:
        food.append(0)

df_new['sun'] = sun
df_new['food'] = food

In [ ]:
print(df_new['db_course'].unique())
display(df_new)

In [ ]:
df_new.loc[ df_new['stress'] <= 10, 'stress'] = 0
df_new.loc[(df_new['stress'] > 10) & (df_new['stress'] <= 20), 'stress'] = 1
df_new.loc[(df_new['stress'] > 20) & (df_new['stress'] <= 30), 'stress'] = 2
df_new.loc[(df_new['stress'] > 30) & (df_new['stress'] <= 40), 'stress'] = 3
df_new.loc[(df_new['stress'] > 40) & (df_new['stress'] <= 50), 'stress'] = 4
df_new.loc[(df_new['stress'] > 50) & (df_new['stress'] <= 60), 'stress'] = 5
df_new.loc[(df_new['stress'] > 60) & (df_new['stress'] <= 70), 'stress'] = 6
df_new.loc[(df_new['stress'] > 70) & (df_new['stress'] <= 80), 'stress'] = 7
df_new.loc[(df_new['stress'] > 80) & (df_new['stress'] <= 90), 'stress'] = 8
df_new.loc[ df_new['stress'] > 90, 'stress'] = 9

In [ ]:
# add column to count how many courses were followed
followed_courses = []
for index, row in df_new.iterrows():
    total = 0
    if row['ml_course'] == 1:
        total += 1
    elif row['ml_course'] == 0:
        total -= 1

    if row['ir_course'] == 1:
        total += 1
    elif row['ir_course'] == 0:
        total -= 1

    if row['stat_course'] == 1:
        total += 1
    elif row['stat_course'] == 0:
        total -= 1

    if row['db_course'] == 1:
        total += 1
    elif row['db_course'] == 0:
        total -= 1

    followed_courses.append(total/4)

# replace old column by cleaned one
df_new['followed_courses'] = followed_courses


In [ ]:
display(df_new)
import math

stress_values = []
for index, row in df_new.iterrows():
    if math.isnan(row['stress']):
        stress_values.append(0)
    else:
        stress_values.append(row['stress'])
        
df_new['stress'] = stress_values
        
for index, row in df_new.iterrows():
    if math.isnan(row['stress']):
        print('niet gelukt')

### Explore

In [ ]:
import os
if not os.path.exists('results/odi/'):
    os.makedirs('results/odi/')
    
from collections import Counter


def show_save_histogram(df, column_name):
    """ Makes and saves a histogram of a specific df column. """
    df.hist(column=column_name)
    plt.savefig("results/odi/" + column_name + ".png")



def make_bar_chart(df, column_name):
    """ Makes a bar chart given a data frame column. """
    
    freqs = Counter(df[column_name])
    xvals = range(len(freqs.values()))
    
    plt.figure()
    plt.bar(xvals, freqs.values())
    plt.xticks(xvals, freqs.keys())
    
    plt.savefig("results/odi/" + column_name + ".png")

show_save_histogram(df_new, "age")


make_bar_chart(df, "gender")


In [ ]:
# Descriptives

# Age
mean_age = df_new.loc[:,"age"].mean()
print(f"mean age: {mean_age}")
std_age = df_new.loc[:,"age"].std()
print(f"standard deviation age: {std_age}")

# Neighbors
mean_neighbors = df_new.loc[:,"neighbors"].mean()
print(f"mean neighbors: {mean_neighbors}")
std_neighbors = df_new.loc[:,"neighbors"].std()
print(f"standard deviation neighbors: {std_neighbors}")

# Stress
mean_stress = df_new.loc[:,"stress"].mean()
print(f"mean stress: {mean_stress}")
std_stress = df_new.loc[:,"stress"].std()
print(f"standard deviation stress: {std_stress}")

# Random number
mean_random = df_new.loc[:,"random_number"].mean()
print(f"mean random: {mean_random}")
std_random = df_new.loc[:,"random_number"].std()
print(f"standard deviation random: {std_random}")

# Correlations

In [ ]:
clean_stand = []
for s in df_new['standed_up']:
    if s == 'no':
        clean_stand.append(0)
    elif s == 'yes':
        clean_stand.append(1)
    else:
        clean_stand.append(np.nan)
df_new['standed_up'] = clean_stand
print(df_new['standed_up'].unique())


In [ ]:
c_1 = 0
c_0 = 0
for index, row in df_new.iterrows():
    if row['program'] == "CLS":
        if row['ml_course'] == 0:
            c_0 +=1
        elif row['ml_course'] == 1:
            c_1 +=1
print(f"1 = {c_1}")
print(f"0 = {c_0}")

In [ ]:
times = []
for time in df_new['bedtime_yesterday']:
    if time == time:
        if time.hour == 19 or time.hour == 21 or time.hour == 22:
            times.append(time.hour)
        elif time.hour == 23 or time.hour == 11:
            times.append(23)
        elif time.hour == 0 or time.hour == 12:
            times.append(24)
        elif time.hour == 1 or time.hour == 13:
            times.append(25)
        elif time.hour == 2:
            times.append(26)
        elif time.hour == 15 or time.hour == 3:
            times.append(27)
        elif time.hour == 4 or time.hour == 16:
            times.append(28)
        elif time.hour == 5:
            times.append(29)
    else:
        times.append(np.nan)
df_new['bedtime_yesterday'] = times

print(df_new['bedtime_yesterday'].unique())


In [ ]:
# cov3_df = pd.DataFrame()
# cov3_df['stress'] = df_new['stress']
# cov3_df['bedtime'] = [float(n) for n in df_new['bedtime_yesterday']]
# cov3_df.cov()
# # plt.plot(cov_df['stress'], cov_df['nr'])
# # display(cov2_df)


In [ ]:
import scipy.stats as scs

def categories(series):
    return range(int(series.min()), int(series.max()) + 1)

# ml_course & program > chi square
df_col1, df_col2 = df_new['gender'], df_new['standed_up']

result = [[sum((df_col1 == cat1) & (df_col2 == cat2)) for cat2 in categories(df_col2)] 
          for cat1 in categories(df_col1)]

chi2_stat, p_val, dof, ex = scs.chi2_contingency(result)
print("===Chi2 Stat===")
print(chi2_stat)
print("\n")
print("===Degrees of Freedom===")
print(dof)
print("\n")
print("===P-Value===")
print(p_val)
print("\n")
print("===Contingency Table===")
print(ex)

### Explore correlations with gender

In [ ]:
# Average stress for each gender
ax = plt.subplot()
ax.set_ylabel('Average stress levels')
ax.set_title('Average stress levels for each gender')
df_new.groupby('gender').mean()['stress'].plot(kind='bar', 
                                               figsize=(5, 6), 
                                               ax = ax,
                                               color="maroon");

In [ ]:
# Average age for each gender
ax = plt.subplot()
ax.set_ylabel('Average age')
ax.set_title('Average age for each gender')
df_new.groupby('gender').mean()['age'].plot(kind='bar', 
                                               figsize=(5, 6), 
                                               ax = ax,
                                               color="maroon");

In [ ]:
# Convert chocolate categories
try:
    df_new['gender'] = df_new['gender'].map( {'female': 0, 'male': 1, 'unknown': 2} ).astype(int)
    df_new['chocolate'] = df_new['chocolate'].map( {'fat': 0, 'neither': 1, 'I have no idea what you are talking about':2, 'slim':3, 'unknown': 4} ).astype(int)
except ValueError:
    print("already converted to int")
    
col_male, col_female = [], []
for index, row in df_new.iterrows():
    if row["gender"] == 1:
        col_male.append(1)
    else:
        col_male.append(0)
    
    if row["gender"] == 0:
        col_female.append(1)
    else:
        col_female.append(0)
    
try:
    df_new.insert(loc=0, column='male', value=pd.Series(col_male))
    df_new.insert(loc=0, column='female', value=pd.Series(col_female))   
except ValueError:
    print("Already added col")


plot = df_new.groupby('program').agg('sum')[['female', 'male']].plot(kind='bar', 
                                                                    fontsize=14, 
                                                                    figsize=(15, 6),
                                                                    stacked=True, 
                                                                    color=['black', 'grey'],
                                                                    title="program based on gender");

In [ ]:
plot = df_new.groupby('chocolate').agg('sum')[['female', 'male']].plot(kind='bar', 
                                                                    fontsize=14, 
                                                                    figsize=(5, 6),
                                                                    stacked=True, 
                                                                    color=['black', 'grey'],
                                                                    title="program based on gender");

In [ ]:
plot = df_new.groupby('stress').agg('sum')[['female', 'male']].plot(kind='bar', 
                                                                    fontsize=14, 
                                                                    figsize=(15, 6),
                                                                    stacked=True, 
                                                                    color=['black', 'grey'],
                                                                    title="program based on gender");

In [ ]:
plot = df_new.groupby('age').agg('sum')[['female', 'male']].plot(kind='bar', 
                                                                    fontsize=14, 
                                                                    figsize=(12, 6),
                                                                    stacked=False, 
                                                                    color=['black', 'grey'],
                                                                    title="program based on gender");

In [ ]:
df_new.groupby('gender')['age'].mean() # 0 = female, 1 = male, 2 = unknown



In [ ]:
df_new.groupby('age').agg('sum')[['female', 'male']]

In [ ]:
display(df_new)

### Regression

In [ ]:
# train & return a classifier
def train_regression(frame):
    # independent variables: ml_course, ir_course, stat_course, db_course
    X = frame['followed_courses'].values.reshape(-1,1)
    
    # dependent variable: stress category 0-9
    Y = frame['stress'].values.reshape(-1,1)
    
    # fit a linear model
    lr = LinearRegression()
    lr.fit(X,Y)
    
    return lr
    
# use a given classifier and test its performance, return a performance measure
def test_regression(frame, classifier):
    X = frame['followed_courses'].values.reshape(-1,1)
    Y = frame['stress'].values.reshape(-1,1)
    Y_pred = classifier.predict(X)
    
    # visualize
    plt.scatter(X,Y)
    plt.plot(X,Y_pred,color='red')
    plt.show()
    
    # compute mean squared error
    mse = mean_squared_error(Y, Y_pred)

    return mse

# k-fold cross validation, takes as an argument a pandas dataframe
def kfold(df):

    # determine amount of folds
    k = 4

    # keep track of created datasets
    folds = []
        
    # substract k subsets
    for i in range(k):
        drop_indices = np.random.choice(df.index, int((1/k)*len(df)), replace=False)
        folds.append(df.drop(drop_indices))
        df.drop(drop_indices, inplace=True)

    # keep track of performances
    performances = []
    
    # for each subset (fold)
    for j in range(len(folds)):

        # take this fold as test set
        test_data = folds[j]

        # take remaining groups as training set
        train_dataframes = []
        
        for k in range(len(folds)):
            if k != j:
                train_dataframes.append(folds[k])
                
        train_data = pd.concat(train_dataframes)

        # train & test regression
#         solution = train_regression(train_data)
#         performances.append(test_regression(test_data, solution))
        
        # train & test 1R
        solution = train_weighted_1R(train_data)
        performances.append(test_weighted_1R(test_data, solution))
        
    print(performances)
    # compute overall performance
    return statistics.mean(performances)

kfold(df_new)

In [ ]:
def train_weighted_1R(frame):
    weights = {'CLS':{'T':0,'Y':0,'P':0}, 'AI':{'T':0,'Y':0,'P':0}, 'CS':{'T':0,'Y':0,'P':0},
               'QRM':{'T':0,'Y':0,'P':0}, 'OTHER':{'T':0,'Y':0,'P':0}}
    
    for index, row in frame.iterrows():
        weights[row['program']]['T'] += 1
        if row['ml_course'] == 1:
            weights[row['program']]['Y'] += 1
    
    for key, value in weights.items():
        if value['T'] != 0:
            value['P'] = value['Y']/value['T']
        
    return weights

def test_weighted_1R(frame,weights):
    predictions = []
    
    for index,row in frame.iterrows():
        p_value = weights[row['program']]['P']
        if random.uniform(0,1) < p_value:
            predictions.append(1)
        else:
            predictions.append(0)

    good_predictions = 0
    i = 0
    for index,row in frame.iterrows():
        if predictions[i] == row['ml_course']:
            good_predictions += 1
        i += 1
        
    print(f"performance = good predictions ({good_predictions}) / total({len(predictions)})")
    performance = good_predictions / len(predictions)
    
    # performance is percentage correctly predicted
    return performance

In [ ]:
counter = 0
for index, row in df_new.iterrows():
    if row['ml_course'] == 0:
        counter += 1
print(counter)


In [ ]:
def regression(frame):
    
    # independent variables: ml_course, ir_course, stat_course, db_course
    X = frame['followed_courses'].values.reshape(-1,1)
    
    # dependent variable: stress category 0-9
    Y = frame['stress'].values.reshape(-1,1)
    
    # fit a linear model
    lr = LinearRegression()
    lr.fit(X,Y)
    
    Y_pred = lr.predict(X)
    
    # visualize
    plt.scatter(X,Y)
    plt.plot(X,Y_pred,color='red')
    plt.show()

regression(df_new)
#print(df_new['stress'].unique())
#print(df_new['followed_courses'].unique())